In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
!pip install pyspark

In [26]:
from pyspark.sql import SparkSession

In [27]:
spark = SparkSession.builder.appName("appName").getOrCreate()

In [28]:
df = spark.read.csv("../input/titanic/train.csv", inferSchema=True, header=True)

In [29]:
schema_df = df.schema

In [35]:
df = spark.readStream.format("csv").schema(schema_df).option("header", True).load("../input/titanic/trai*.csv")

In [36]:
df.explain()

In [37]:
result = df.groupBy("Sex").sum("Survived")
display(result)

In [38]:
result.explain()

In [39]:
streamresult = df.writeStream.format("csv")\
                                    .outputMode("append")\
                                    .option("path", "../writer/")\
                                    .option("checkpointLocation", "../writerStream")\
                                    .start()\
                                    .awaitTermination()

In [42]:
!ls ../writer/